#

# input layerの入力表現

## 流れ

1. パッチへ分割
   1. 画像をパッチに分割する
   2. flattenして、パッチの縦×横×チャンネルの長さのベクトルに変換する
2. パッチ埋め込み
   1. 1層の線形層で良いベクトルに変換する
   2. 良いベクトル:=損失が少ないベクトル
3. クラストークンを定義
   1. 画像全体の情報を保持する
   2. パッチ埋め込みのベクトルの大きさのベクトル
   3. 標準正規分布に従った値を設定
   4. クラストークンは学習のパラメータ
4. 位置埋め込み
   1. パッチ埋め込みだけではパッチの位置情報がないため、位置をクラストークンとパッチ埋め込みベクトルに埋め込む
   2. 初期値は標準正規分布に従う乱数を設定


## 数式表現

### 1. 入力画像からflattenまで

> $H$: 画像の高さ（pixel）
>
> $W$: 画像の幅（pixel）
>
> $C$: 色情報（RGB）
>
> $N_p$: パッチの個数 
>
> $P$: パッチの縦横のサイズ（pixel）


とすると入力画像を

> $\boldsymbol{x} \in \mathbb{R}^{H \times W \times C}$


flattenした入力画像を

> $\boldsymbol{x_p} \in \mathbb{R}^{N_p\times(P^2 \cdot C)}$

と表せる。

flattenした各バッチのベクトルは
> $x_p^1, x_p^2, \dots, x_p^{N_p}$

と表す。


### 2. パッチ埋め込み

> $D$ :「$\boldsymbol{x_p}$ を埋め込んだベクトルの次元」

とすると、$\boldsymbol{x_p}$を埋め込む線形層の重みは

> $E \in \mathbb{R}^{(P^2 \cdot C) \times D} $ 

と表せ、各バッチに$E$を適用すると

> $x_{p}^i E \in \mathbb{R}^D \quad (i = 1, 2, \dots, N_p)$ 

であり、

> $x_p E = [ x_p^1 E; x_p^2 E; \dots, x_p^{N_p} E ] \in \mathbb{R}^{N_p \times D}$

である。


### 3. クラストークン


$x_p E$ にクラストークンを付加する。$x_{class} \in \mathbb{R}^D$ を$x_p E$に追加して、

> $x_{p+t} E = [ x_{class}; x_p^1E;  x_p^2E; \dots, x_p^{N_p}E] \in \mathbb{R}^{(N_p + 1) \times D}$

である。

### 4. 位置埋め込み

> トークン数N:「クラストークン+パッチ数」

とすると、トークン数は$N = N_p + 1$である。

位置埋め込みはD次元ベクトルがトークン数、すなわち

> $E_{pos} = [E x_{pos}^{class}; E x_{pos}^1; E x_{pos}^2; \dots; E x_{pos}^{N_p}] \in \mathbb{R}^{(N) \times D}$

であるから、位置埋め込みしたEncoderへの入力$z_0$は

> $z_0 = x_{p+t}E + E_{pos} \in \mathbb{R}^{(N) \times D} $

である。

# Input Layerの実装

In [15]:
import torch
import torch.nn as nn

# emb: 埋め込み。embedded

class VitInputLayer(nn.Module):
    def __init__(
        self,
        in_channels: int = 3,
        emb_dim: int = 384,
        num_patch_row: int = 2,
        image_size: int = 32
    ):
        """        
        Args:
            in_channels (int, optional): 入力画像のチャンネル数. Defaults to 3.
            emb_dim (int, optional): 埋め込み後のベクトルの長さ. Defaults to 384.
            num_patch_row (int, optional): 高さ方向のパッチ数. Defaults to 2.
            image_size (int, optional): 入力画像の1辺の大きさ. Defaults to 32.
        """
        super().__init__()
        self.in_channels = in_channels
        self.emb_dim = emb_dim
        self.num_patch_row = num_patch_row
        self.image_size = image_size

        # パッチ数
        self.num_patch = self.num_patch_row ** 2

        # パッチの大きさ
        self.patch_size = int(self.image_size // self.num_patch_row)

        # 乳画像のパッチへの分割 & パッチ埋め込みを一気に行う層
        self.patch_emb_layer = nn.Conv2d(
            in_channels=self.in_channels,
            out_channels=self.emb_dim,
            kernel_size=self.patch_size,
            stride=self.patch_size
        )

        # クラストークン
        self.cls_token = nn.parameter.Parameter(
                torch.randn(1,1, emb_dim)
            
        )

        # 位置埋め込み
        # トークン数(パッチ数+クラストークン数(1))
        num_token = self.patch_size + 1
        self.pos_emb = nn.parameter.Parameter(
            torch.randn(1, self.num_patch+1, emb_dim)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """前処理

        Args:
            x (torch.Tensor): （B, C, H, W）
                B: バッチサイズ
                C: チャンネル数
                H: 高さ
                W: 幅

        Returns:
            torch.Tensor: ViTへの入力。(B, N, D)
                B: バッチサイズ
                N: トークン数
                D: 埋め込みベクトルの長さ
        """

        # パッチの埋め込み

        ## P: パッチの1辺のサイズ
        ## flattenはパッチ埋め込みの後

        ## パッチ埋め込み (B, C, H, W) -> (B, D, H/P, W/P)
        z_0 = self.patch_emb_layer(x)

        ## flatten (B, D, H/P, W/P) -> (B, D, Np)
        ## Np はパッチ数 (= H*W / P**2)
        z_0 = z_0.flatten(2)

        ## 軸の順番を変更 (B, D, Np) -> (B, Np, D)
        z_0 = z_0.transpose(1, 2)

        # クラストークンを結合 (B, Np, D) -> (B, N, D)
        # cls_token: (1, 1, D) から (B, 1, D)に変換して結合
        z_0 = torch.cat(
            [self.cls_token.repeat(repeats=(x.size(0), 1, 1)), z_0],
            dim=1
        )

        # 位置埋め込み
        z_0 = z_0 + self.pos_emb

        return z_0

batch_size = 2
channel = 3
height = 32
weight = 32

x = torch.randn(batch_size, channel, height, weight)
input_layer = VitInputLayer(num_patch_row=2)
z_0 = input_layer(x)

print(f"(B, N, D) = {z_0.shape}")

(B, N, D) = torch.Size([2, 5, 384])
